In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import MinMaxScaler

In [2]:
df = pd.read_csv('preprocess.csv')
df.head()

,age,sex,presure_blood_resting,colesterol,cigarettes_per_day,smoker_years,sugar,max_heart_rate,res_heart_rate,blood_presure_sistoles,...,major_vessels_1.0,major_vessels_2.0,major_vessels_3.0,electrocardio_0.0,electrocardio_1.0,electrocardio_2.0,chest_pain_1.0,chest_pain_2.0,chest_pain_3.0,chest_pain_4.0
0,63.0,1.0,145.0,233.0,50.0,20.0,1.0,150.0,60.0,190.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1,67.0,1.0,160.0,286.0,40.0,40.0,0.0,108.0,64.0,160.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,67.0,1.0,120.0,229.0,20.0,35.0,0.0,129.0,78.0,140.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,37.0,1.0,130.0,250.0,0.0,0.0,0.0,187.0,84.0,195.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,41.0,0.0,130.0,204.0,0.0,0.0,0.0,172.0,71.0,160.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


In [3]:
n = 50
random_state = 42

X = df.loc[:, df.columns != 'prob']
y = df['prob']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
scaler = MinMaxScaler(feature_range=(-1, 1)).fit(X_train)

# Apply the normalization trained in training data in both training and test sets
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

## Linear

In [4]:
#knc = LinearSVC()
knc = SVC(kernel='linear')
knc.fit(X_train, y_train)
pred = knc.predict(X_test)

print("Confusion matrix on test set:\n", sk.metrics.confusion_matrix(y_test, pred))
print("\nAccuracy on test set: ", sk.metrics.accuracy_score(y_test, pred))
print("\nF1 score on test set: ", sk.metrics.f1_score(y_test, pred, average='micro'))

Confusion matrix on test set:
 [[192   6   5   5   0]
 [ 29   9  12  12   1]
 [  4   7   4  12   0]
 [  4   8   7  16   1]
 [  0   3   4   6   2]]

Accuracy on test set:  0.6389684813753582

F1 score on test set:  0.6389684813753582


In [ ]:
# List of C values to test. We usualy test diverse orders of magnitude
Cs = np.logspace(-3, 6, num=10, base=10.0)
#Cs = np.logspace(-3, 5, num=9, base=10.0)

param_grid = {'C': Cs}
#grid_search = GridSearchCV(LinearSVC(), param_grid, cv=10)
grid_search = GridSearchCV(SVC(kernel='linear'), param_grid, cv=5, n_jobs=-1, scoring='f1_micro')
grid_search.fit(X_train, y_train)

# Let's plot the 10-fold cross.validation accuracy deppending on C
scores = grid_search.cv_results_['mean_test_score']

In [ ]:
plt.semilogx(Cs,scores)
plt.ylabel('F1 score')
plt.xlabel('C')
plt.savefig('../figures/svm_linear_C_cv.pdf')

parval=grid_search.best_params_
cvacc = cross_val_score(SVC(C=parval['C'],kernel='linear'), X=X_train,  y=y_train, cv=5, scoring='f1_micro', n_jobs=-1)
print('F1 score 5-fold cross on train data= ', cvacc.mean())

# Let's apply the best C parameter found to the test set

In [ ]:
#knc = LinearSVC(C=parval['C'])
svcl = SVC(C=parval['C'], kernel='linear')
svcl.fit(X_train, y_train)
pred = svcl.predict(X_test)

print("\nConfusion matrix on test set:\n", sk.metrics.confusion_matrix(y_test, pred))
print("\nAccuracy on test set: ", sk.metrics.accuracy_score(y_test, pred))
print("\nF1 score on test set: ", sk.metrics.f1_score(y_test, pred, average='micro'))
print("\nBest value of parameter C found: ", parval)
print("\nNumber of supports: ", np.sum(svcl.n_support_), "(",np.sum(np.abs(svcl.dual_coef_)==parval['C']) ,"of them have slacks)")
print("Prop. of supports: ", np.sum(svcl.n_support_)/X_train.shape[0])

## Poly

In [ ]:
svcp2 = SVC(kernel='poly', degree=2)
svcp2.fit(X_train, y_train)

pred = svcp2.predict(X_test)
print("Confusion matrix on test set:\n",sk.metrics.confusion_matrix(y_test, pred))
print("\nAccuracy on test set: ",sk.metrics.accuracy_score(y_test, pred))

In [ ]:
Cs = np.logspace(-3, 6, num=10, base=10.0)

param_grid = {'C': Cs}
grid_search = GridSearchCV(SVC(kernel='poly', degree=2) , param_grid, cv=5, n_jobs=-1, scoring='f1_micro')
grid_search.fit(X_train, y_train)

scores = grid_search.cv_results_['mean_test_score']

In [ ]:
plt.semilogx(Cs,scores)
plt.ylabel('F1 score')
plt.xlabel('C')
plt.savefig('../figures/svm_poly_C_cv.pdf')

In [ ]:
parval = grid_search.best_params_

cvacc = cross_val_score(SVC(kernel='poly', degree=2, C=parval['C']) , X=X_train,  y=y_train, cv=5, scoring='f1_micro', n_jobs=-1)
print('F1 score 5-fold cross on train data= ', cvacc.mean())

svcp2 = SVC(kernel='poly', degree=2, C=parval['C'])
svcp2.fit(X_train, y_train)
pred = svcp2.predict(X_test)

In [ ]:
print("\nConfusion matrix on test set:\n", sk.metrics.confusion_matrix(y_test, pred))
print("\nAccuracy on test set: ", sk.metrics.accuracy_score(y_test, pred))
print("\nF1 score on test set: ", sk.metrics.f1_score(y_test, pred, average='micro'))
print("\nBest combination of parameters found: ",parval)
print("\nNumber of supports: ",np.sum(svcp2.n_support_), "(",np.sum(np.abs(svcp2.dual_coef_)==parval['C']) ,"of them have slacks)")
print("Prop. of supports: ",np.sum(svcp2.n_support_)/X_train.shape[0])

## Poly3

In [ ]:
svcp3 = SVC(kernel='poly', degree=3)
svcp3.fit(X_train, y_train)
pred = svcp3.predict(X_test)

print("Confusion matrix on test set:\n", sk.metrics.confusion_matrix(y_test, pred))
print("\nAccuracy on test set: ", sk.metrics.accuracy_score(y_test, pred))

In [ ]:
Cs = np.logspace(-3, 6, num=10, base=10.0)

param_grid = {'C': Cs}
grid_search = GridSearchCV(SVC(kernel='poly', degree=3) , param_grid, cv=5, n_jobs=-1, scoring='f1_micro')
grid_search.fit(X_train,y_train)

scores = grid_search.cv_results_['mean_test_score']

In [ ]:
plt.semilogx(Cs, scores)
plt.ylabel('F1 score')
plt.xlabel('C')
plt.savefig('../figures/svm_poly3_C_cv.pdf')

In [ ]:
parval = grid_search.best_params_

cvacc = cross_val_score(SVC(kernel='poly', degree=3, C=parval['C']) , X=X_train,  y=y_train, cv=5, scoring='f1_micro', n_jobs=-1)
print('F1 score 5-fold cross on train data= ', cvacc.mean())

svcp3 = SVC(kernel='poly', degree=3, C=parval['C'])
svcp3.fit(X_train, y_train)
pred = svcp3.predict(X_test)

In [ ]:
print("\nConfusion matrix on test set:\n", sk.metrics.confusion_matrix(y_test, pred))
print("\nAccuracy on test set: ", sk.metrics.accuracy_score(y_test, pred))
print("\nF1 score on test set: ", sk.metrics.f1_score(y_test, pred, average='micro'))
print("\nBest combination of parameters found: ", parval)
print("\nNumber of supports: ",np.sum(svcp3.n_support_), "(",np.sum(np.abs(svcp3.dual_coef_)==parval['C']) ,"of them have slacks)")
print("Prop. of supports: ",np.sum(svcp3.n_support_)/X_train.shape[0])

## RBF Kernel

In [ ]:
svcrbf = SVC()
svcrbf.fit(X_train, y_train)
pred = svcrbf.predict(X_test)
print("Confusion matrix on test set:\n", sk.metrics.confusion_matrix(y_test, pred))
print("\nAccuracy on test set: ", sk.metrics.accuracy_score(y_test, pred))

In [ ]:
# Values we will test for each parameter. When observing results, consider the limits of the
# values tested and increase them if necessary
gammas = [0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10]
Cs = np.logspace(-1, 6, num=8, base=10.0)

param_grid = {'C': Cs, 'gamma' : gammas}
grid_search = GridSearchCV(SVC(), param_grid, cv=5, n_jobs=-1, scoring='f1_micro')
grid_search.fit(X_train,y_train)
parval=grid_search.best_params_

# We'll show in a grid, the accuracy for each combination of parameters tester
scores = grid_search.cv_results_['mean_test_score']
scores = np.array(scores).reshape(len(param_grid['C']), len(param_grid['gamma']))

In [ ]:
plt.matshow(scores)
plt.xlabel('gamma')
plt.ylabel('C')
plt.colorbar(label='F1 score')
plt.grid(b=None)
plt.xticks(np.arange(len(param_grid['gamma'])), param_grid['gamma'], rotation='vertical')
plt.yticks(np.arange(len(param_grid['C'])), param_grid['C'])
plt.savefig("../figures/svm_rbf_C_cv.pdf")

parval = grid_search.best_params_
print("\nBest combination of parameters found: ", parval)

cvacc = cross_val_score(SVC(C=parval['C'], gamma=parval['gamma']) , X=X_train,  y=y_train, cv=5, scoring='f1_micro', n_jobs=-1)
print('\nF1 score 5-fold cross on train data= ', cvacc.mean())

In [ ]:
svcrbf = SVC(C=parval['C'], gamma=parval['gamma'])
svcrbf.fit(X_train, y_train)
pred = svcrbf.predict(X_test)

print("Confusion matrix on test set:\n", sk.metrics.confusion_matrix(y_test, pred))
print("\nAccuracy on test set: ", sk.metrics.accuracy_score(y_test, pred))
print("\nF1 score on test set: ", sk.metrics.f1_score(y_test, pred, average='micro'))
print("\nNumber of supports: ",np.sum(svcrbf.n_support_), "(",np.sum(np.abs(svcrbf.dual_coef_)==parval['C']) ,"of them have slacks)")
print("Prop. of supports: ",np.sum(svcrbf.n_support_)/X_train.shape[0])